# Image retrieval, captioning and classification with CoCo

This tutorial uses the [CoCo dataset "Common objects in Context"](https://cocodataset.org/#home) to show case some of the key-features of SuperDuperDB. In this example, you'll learn how to:

- Prepare data in the best way for SuperDuperDB usage
- Define data types
- Upload and query data to and from the data base
- Define multiple models on the database, including models with dependencies
- Define a searchable semantic index based on existing models
- Train a semantic index from scratch

If you haven't downloaded the data already, execute the lines of bash below. We've tried to keep it clean,
and for reasons of efficiency have resized the images using imagemagick.

In [ ]:
!mkdir -o data/coco/
!curl http://images.cocodataset.org/annotations/annotations_trainval2014.zip -o data/coco/raw.zip
!unzip data/coco/raw.zip
!mv data/coco/annotations/captions_train2014.json data/coco/
!rm -rf data/coco/annotations
!rm data/coco/raw.zip
!curl http://images.cocodataset.org/zips/train2014.zip -o data/coco/images.zip
!unzip data/coco/images.zip
!rm data/coco/images.zip
!sudo apt install imagemagick
!mogrify -resize 224x data/coco/images/*.jpg

SuperDuperDB uses MongoDB for data storage. If you haven't done so already, install it using the following lines of bash.

In [ ]:
!wget -qO - https://www.mongodb.org/static/pgp/server-6.0.asc | sudo apt-key add -
!sudo apt-get install gnupg
!wget -qO - https://www.mongodb.org/static/pgp/server-6.0.asc | sudo apt-key add -
!echo "deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu focal/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list
!sudo apt-get update
!sudo apt-get install -y mongodb-org

In case you haven't done so already, install the dependencies for this tutorial, including SuperDuperDB,
which is a simple pip install.

In [ ]:
!pip install pandas
!pip install pillow
!pip install torch
!pip install superduperdb

SuperDuperDB can handle data in any format, including images. The documents in the database are MongoDB `bson` documents, which mix `json` with raw bytes and `ObjectId` objects. SuperDuperDB takes advantage of this by 
serializing more sophisticated objects to bytes, and reinstantiating the objects in memory, when data is queried.

In order to tell SuperDuperDB what type an object has, one specifies this with a subdocument of the form:

```json
{
    "_content": {
        "bytes": ...,
        "type": "<my-type>",
    }
}
```

If however, the content is located on the web or the filesystem, one can specify the URLs directly:

```json
{
    "_content": {
        "url": "<url-or-file>",
        "type": "<my-type>",
    }
}
```

Let's see this now in action. We reformat the CoCo data, so that each image is associated in one document with all of the captions which describe it, and add the location of the images using the `_content` formalism.

In [ ]:
import json

with open('data/coco/captions_train2014.json') as f:
    raw = json.load(f)
    
raw['images'] = {x['id']: x for x in raw['images']}

for im in raw['images']:
    raw['images'][im]['captions'] = []
    
for a in raw['annotations']:
    raw['images'][a['image_id']]['captions'].append(a['caption'])

raw = list(raw['images'].values())

for i, im in enumerate(raw):
    # if image is already in memory, then add 'bytes': b'...' instead of 'url': '...'
    # for content located on the web, use 'http://' or 'https://' instead of 'file://'
    im['img'] = {
        '_content': {'url': f'file://data/coco/images/{im["file_name"]}', 'type': 'image'}
    }
    raw[i] = {'captions': im['captions'], 'img': im['img']}

with open('data/coco/data.json', 'w') as f:
    json.dump(raw, f)

In [ ]:
import json
import sys

sys.path.append('../../')

from superduperdb.client import the_client
from IPython.display import display, clear_output
import torch

docs = the_client.coco_example.documents

We'll load the data and add most of it to the database. We'll hold back some data so that we can see how to update 
the database later.

In [ ]:
with open('data/coco/data.json') as f:
    data = json.load(f)
    
docs.insert_many(data[:-1000]), verbose=True)

We previously added the type `image` to the `_content` subrecords earlier.
So that we can load the data using this type, we need to add this type to the database.
You can see in `examples/types.py` how the class encodes and decodes data. Suffice to say at this point, 
that each type has an `encode` and `decode` method, which convert to and from `bytes`.

In [ ]:
from examples.types import FloatTensor, Image

docs.create_type('float_tensor', FloatTensor())
docs.create_type('image', Image())

In the first AI task which we implement for the `docs` collection, we'll be setting up a model to retrieve relevant images using provided text. For this data, that means the `captions` field being used to retrieve the `img` field. In order to be able to keep an objective record of performance, we can set up an immutable validation dataset from the collection. We use a **splitter** to define how we'd like to test retrieval. This splits the documents into query and retrieved document.

In [ ]:
docs.create_validation_set(
    'text2image_retrieval', 
    filter={},
    splitter=lambda x: ({'img': x['img']}, {'captions': [x['captions'][0]]}),
    sample_size=1000,
)

We can see what the data points in the validation set look like by querying:

In [ ]:
docs['_validation_sets'].find_one({'_validation_set': 'text2image_retrieval'})

You can see that the sample "query" is split into the `_other` field. This is important when evaluating semantic indexes.

Now let's start adding a model to the collection.
A nice open source model to test text-2-image retrieval is [CLIP](https://openai.com/blog/clip/) which understands images and texts and embeds these in a common vector space.

Note that we are specifying the type of the model output, so that the collection knows how to store the results, as well as "activating" the model with `active=True`. That means, whenever we add data which fall under the `filter`, then these will get processed by the model, and the outputs will be added to the collection documents.

The `key` argument specifies which part of the document the model should act. If `key="_base"` then the model takes the whole document as input. Since we'll be encoding documents as images, then we'll chose `key="img`.

In [ ]:
from examples.models import CLIP

docs.create_model(
    name='clip',
    object=CLIP('RN50'),
    filter={},
    type='float_tensor',
    key='img',
    verbose=True,
    active=True
)

We'll create a companion model which uses the same underlying object as the previous model. That's specified by adding the name instead of the object in the `object` argument. In this case the model is not `active`, since we'll only be using it for querying the collection. We don't need to specify a `type` since that was done in the last step.

In [ ]:
docs.create_model(
    name='clip_text',
    object='clip',
    key='captions',
    active=False,
)

We'll also create a measure which tests how similar to each other two outputs might be. Since CLIP was trained with cosine-similarity we'll use that here too.

In [ ]:
from examples.measures import css

docs.create_measure('css', css)

In order to be able to measure performance on the validation set, we'll add a **metric**.

In [ ]:
from examples.metrics import PatK

docs.create_metric('p_at_10', PatK(10))

Now we're ready to go to add a **semantic index**. This is a tuple of models, one of which is activated in order to populate the collection with vectors. The idea is that any of the models in the **semantic index** can be used to query the collection using nearest neighbour lookup based on the **measure** chosen.

In [ ]:
from examples.models import CLIP

docs.create_semantic_index(
    'clip',
    models=['clip', 'clip_text'],
    measure='css',
    metrics=['p_at_10'],
)

In [ ]:
from bson import ObjectId
from IPython.display import display

docs.semantic_index = 'clip'
for r in docs.find({'$like': {'document': {'_id': ObjectId('63d27372745cc274ef3518f2')}, 'n': 10}}):
    display(r['img'])

Let's now evaluate the quality of this semantic index

In [ ]:
docs.validate_semantic_index('clip', ['text2image_retrieval'], ['p_at_10'])

In [ ]:
docs['_semantic_indexes'].find_one()

We can see that we can get nice meaningful retrievals using the CLIP model from short descriptive pieces of text.
This is very useful, since the model is now deployed to the database, listening for incoming queries.

In [ ]:
docs.list_semantic_indexes()

In [ ]:
from IPython.display import display

docs.semantic_index = 'clip'
for r in docs.find({'$like': {'document': {'captions': ['Dog catches a frisbee']}, 'n': 5}}):
    display(r['img'])

In the next section of this example, let us train our own model from scratch. The model will be much simpler than the clip model, but will yield faster retrievals. It will be interesting to see how this compares to CLIP, and show-case SuperDuperDB as a framework for easily integrating and benchmarking AI models, in particular for retrieval.

First we will implement a simpler sentence embedding, using a simple word-embedding approach based around Glove.
Please look at the model in `examples.models.AverageOfGloves`.

In [ ]:
!curl https://nlp.stanford.edu/data/glove.6B.zip -o data/glove.6B.zip
!unzip data/glove.6B.zip

We may register this model to the collection in the same way we did for the textual part of CLIP:

In [ ]:
import numpy
import torch
from examples.models import AverageOfGloves

with open('data/glove.6B/glove.6B.50d.txt') as f:
    lines = f.read().split('\n')
    
lines = [x.split(' ') for x in lines[:-1]]
index = [x[0] for x in lines]
vectors = [[float(y) for y in x[1:]] for x in lines]
vectors = numpy.array(vectors)

glove = AverageOfGloves(torch.from_numpy(vectors).type(torch.float), index)

In [ ]:
docs.create_model(
    'average_glove',
    object=glove,
    key='captions',
    active=False,
)

In [ ]:
docs.create_model(
    'clip_projection',
    object=torch.nn.Linear(1024, 50),
    active=True,
    key='img',
    type='float_tensor',
    features={'img': 'clip'},
    verbose=True,
)

Let's also create a loss function, in order to be able to perform the learning task:

In [ ]:
from examples.losses import ranking_loss

docs.create_loss('ranking_loss', ranking_loss)

A semantic index training requires:

- 1 or more models
- A measure function to measure similarity between model outputs
- A loss function
- One or more validation sets
- One or more metrics to measure performance

We now have all of these things ready and registered with the database, so we can start the training:

In [ ]:
docs.create_semantic_index(
    'simple_image_search',
    models=['clip_projection', 'average_glove'],
    loss='ranking_loss',
    filter={},
    projection={'image': 0, '_like': 0},
    metrics=['p_at_10'],
    measure='css',
    validation_sets=['text2image_retrieval'],
    batch_size=250,
    num_workers=0,
    n_epochs=20,
    lr=0.001,
    log_weights=True,
    download=True,
    validation_interval=50,
    no_improve_then_stop=5,
    n_iterations=5000,
    use_grads={'clip_projection': True, 'average_glove': False},
)

We now can see that we've set and trained our own semantic index. Let's take a look:

In [ ]:
docs.list_semantic_indexes()

In [ ]:
from matplotlib import pyplot as plt
info = docs['_semantic_indexes'].find_one({'name': 'simple_image_search'})

In [ ]:
for k in info['metric_values']:
    if k == 'loss':
        print(info['metric_values'][k])
        plt.figure()
        plt.title('loss')
        plt.plot(info['metric_values'][k])
        continue
    for result in info['metric_values'][k]:
        plt.figure()
        plt.title(f'{k}/{result}')
        plt.plot(info['metric_values'][k][result])
plt.show()

In [ ]:
for parameter in info['weights']:
    plt.figure()
    plt.title(parameter)
    plt.plot(info['weights'][parameter])

In [ ]:
docs.list_models()

In [ ]:
docs.refresh_model('clip_projection')

In [ ]:
from IPython.display import display

docs.semantic_index = 'simple_image_search'
for r in docs.find({'$like': {'document': {'captions': ['Dog catches frisbee']}, 'n': 5}}):
    display(r['img'])

In [ ]:
from examples.models import NounWords
docs.create_model('noun_words', NounWords(), verbose=True, key='captions')

In [ ]:
docs.create_validation_set('attribute_prediction', sample_size=250)

In [ ]:
import collections
import tqdm
all_nouns = []
for r in tqdm.tqdm(docs.find({'_fold': 'train'}, {'_outputs.captions.noun_words': 1}), total=docs.count_documents({})):
    all_nouns.extend(r['_outputs']['captions']['noun_words'])
    
counts = dict(collections.Counter(all_nouns))
all_nouns = [w for w in counts if counts[w] > 30]
total = docs.count_documents({})
pos_weights = [counts[w] / total for w in all_nouns]

In [ ]:
from examples.models import FewHot, TopK
from examples.metrics import jacquard_index

docs.create_model('nouns_to_few_hot', FewHot(all_nouns))
docs.create_postprocessor('top_5', TopK(all_nouns, 5))
docs.create_forward('attribute_predictor', torch.nn.Linear(1024, len(all_nouns)))
docs.create_loss('nouns_loss', torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weights)))
docs.create_metric('jacquard_index', jacquard_index)

In [ ]:
from examples.models import FewHot
docs.create_model('nouns_to_few_hot', FewHot(post.tokens), active=False,
                 key='_outputs.captions.noun_words')

In [ ]:
docs.create_model('attribute_predictor', forward='attribute_predictor', postprocessor='top_5',
                  key='img', features={'img': 'clip'})

Let's test the model, using the `apply_model` method:

In [ ]:
docs.apply_model('attribute_predictor', docs.find_one())

In [ ]:
docs.create_imputation(
    'noun_prediction',
    model='attribute_predictor',
    target='nouns_to_few_hot',
    loss='nouns_loss',
    metrics=['jacquard_index'],
    validation_sets=['attribute_prediction'],
    lr=0.001,
    validation_interval=10,
    n_iterations=20,
)

We can view the results of learning (metrics, loss etc.) by looking in the `_imputations` subcollection:

In [1]:
import json
import sys

sys.path.append('../../')

from superduperdb.client import the_client
from IPython.display import display, clear_output
import torch

docs = the_client.coco_example.documents

In [2]:
import tqdm 

all_captions = []
n = docs.count_documents({'_fold': 'train'})
for r in tqdm.tqdm_notebook(docs.find({'_fold': 'train'}, {'captions': 1, '_id': 0}), total=n):
    all_captions.extend(r['captions'])

  0%|          | 0/78560 [00:00<?, ?it/s]

In [3]:
import collections
import re
all_captions = [re.sub('[^a-z ]', '', x.lower()).strip() for x in all_captions]
words = ' '.join(all_captions).split(' ')
counts = dict(collections.Counter(words))
vocab = sorted([w for w in counts if counts[w] > 5 and w])

In [6]:
from examples.models import ConditionalLM, SimpleTokenizer
tokenizer = SimpleTokenizer(vocab)
m = ConditionalLM(tokenizer)

In [7]:
docs.create_model('conditional_lm', object=m, active=False, features={'img': 'clip'}, key='_base')
docs.create_model('captioning_tokenizer', tokenizer, key='caption', active=False)

In [4]:
docs.delete_model('conditional_lm', force=True)
docs.delete_model('captioning_tokenizer', force=True)

You are about to delete these models: [ObjectId('63ebdc062c13085e82d849ce')], are you sure? [y/N]: y
unsetting output field _outputs._base.conditional_lm
You are about to delete these models: [ObjectId('63ebdc062c13085e82d84a6d')], are you sure? [y/N]: y
unsetting output field _outputs.caption.captioning_tokenizer


In [ ]:
from examples.losses import auto_regressive_loss
docs.create_loss('autoregressive_loss', auto_regressive_loss)

In [ ]:
docs['_validation_sets'].find_one({'_validation_set': 'attribute_prediction'})

In [ ]:
from examples.splitters import captioning_splitter

docs.create_splitter('captioning_splitter', captioning_splitter)

In [ ]:
captioning_splitter(docs.find_one())

In [ ]:
docs.create_validation_set('captioning', splitter=docs.splitters['captioning_splitter'],
                           sample_size=500, chunk_size=100)

In [ ]:
docs['_validation_sets'].find_one({'_validation_set': 'captioning'})

In [ ]:
docs.create_imputation(
    'image_captioner',
    model='conditional_lm',
    loss='autoregressive_loss',
    target='captioning_tokenizer',
    splitter='captioning_splitter',
    validation_sets=['captioning'],
    batch_size=50,
    lr=0.001,
)

downloading ids for {'_fold': 'train'}


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78560/78560 [00:00<00:00, 212987.97it/s]


downloading records for {'_fold': 'valid'}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4223/4223 [00:00<00:00, 10464.52it/s]


fold: VALID; iteration: 0; epoch: 0; loss: 9.006699079625747; 
fold: TRAIN; iteration: 0; epoch: 0; loss: 9.007852554321289; 
fold: TRAIN; iteration: 1; epoch: 0; loss: 8.673001289367676; 
fold: TRAIN; iteration: 2; epoch: 0; loss: 8.33862590789795; 
fold: TRAIN; iteration: 3; epoch: 0; loss: 7.989694118499756; 
fold: TRAIN; iteration: 4; epoch: 0; loss: 7.5262064933776855; 
fold: TRAIN; iteration: 5; epoch: 0; loss: 7.103574752807617; 
fold: TRAIN; iteration: 6; epoch: 0; loss: 6.254573822021484; 
fold: TRAIN; iteration: 7; epoch: 0; loss: 5.6371636390686035; 
fold: TRAIN; iteration: 8; epoch: 0; loss: 5.411647319793701; 
fold: TRAIN; iteration: 9; epoch: 0; loss: 4.917050838470459; 
fold: TRAIN; iteration: 10; epoch: 0; loss: 5.1513471603393555; 
fold: TRAIN; iteration: 11; epoch: 0; loss: 5.066558361053467; 
fold: TRAIN; iteration: 12; epoch: 0; loss: 4.722397804260254; 
fold: TRAIN; iteration: 13; epoch: 0; loss: 5.1696906089782715; 
fold: TRAIN; iteration: 14; epoch: 0; loss: 5.10

...

fold: TRAIN; iteration: 5086; epoch: 3; loss: 2.174572706222534; 
fold: TRAIN; iteration: 5087; epoch: 3; loss: 2.1135478019714355; 
fold: TRAIN; iteration: 5088; epoch: 3; loss: 2.311033248901367; 
fold: TRAIN; iteration: 5089; epoch: 3; loss: 2.3039441108703613; 
fold: TRAIN; iteration: 5090; epoch: 3; loss: 2.297203302383423; 
fold: TRAIN; iteration: 5091; epoch: 3; loss: 2.3317465782165527; 
fold: TRAIN; iteration: 5092; epoch: 3; loss: 2.1797242164611816; 
fold: TRAIN; iteration: 5093; epoch: 3; loss: 2.3144290447235107; 
fold: TRAIN; iteration: 5094; epoch: 3; loss: 2.2920730113983154; 
fold: TRAIN; iteration: 5095; epoch: 3; loss: 2.4444284439086914; 
fold: TRAIN; iteration: 5096; epoch: 3; loss: 2.3666634559631348; 
fold: TRAIN; iteration: 5097; epoch: 3; loss: 2.4668827056884766; 
fold: TRAIN; iteration: 5098; epoch: 3; loss: 2.2540242671966553; 
fold: TRAIN; iteration: 5099; epoch: 3; loss: 2.1780877113342285; 
fold: TRAIN; iteration: 5100; epoch: 3; loss: 2.2957100868225098;

fold: TRAIN; iteration: 5210; epoch: 3; loss: 2.1599597930908203; 
fold: TRAIN; iteration: 5211; epoch: 3; loss: 2.3075335025787354; 
fold: TRAIN; iteration: 5212; epoch: 3; loss: 2.2544543743133545; 
fold: TRAIN; iteration: 5213; epoch: 3; loss: 2.1686599254608154; 
fold: TRAIN; iteration: 5214; epoch: 3; loss: 2.4153943061828613; 
fold: TRAIN; iteration: 5215; epoch: 3; loss: 2.2297403812408447; 
fold: TRAIN; iteration: 5216; epoch: 3; loss: 2.2023003101348877; 
fold: TRAIN; iteration: 5217; epoch: 3; loss: 2.292957067489624; 
fold: TRAIN; iteration: 5218; epoch: 3; loss: 2.3238980770111084; 
fold: TRAIN; iteration: 5219; epoch: 3; loss: 2.2902278900146484; 
fold: TRAIN; iteration: 5220; epoch: 3; loss: 1.9527878761291504; 
fold: TRAIN; iteration: 5221; epoch: 3; loss: 2.1740224361419678; 
fold: TRAIN; iteration: 5222; epoch: 3; loss: 2.1557271480560303; 
fold: TRAIN; iteration: 5223; epoch: 3; loss: 2.4852216243743896; 
fold: TRAIN; iteration: 5224; epoch: 3; loss: 2.241579294204712

fold: TRAIN; iteration: 5334; epoch: 3; loss: 2.137690544128418; 
fold: TRAIN; iteration: 5335; epoch: 3; loss: 2.1437885761260986; 
fold: TRAIN; iteration: 5336; epoch: 3; loss: 2.1641993522644043; 
fold: TRAIN; iteration: 5337; epoch: 3; loss: 2.3727622032165527; 
fold: TRAIN; iteration: 5338; epoch: 3; loss: 2.173328161239624; 
fold: TRAIN; iteration: 5339; epoch: 3; loss: 2.367058753967285; 
fold: TRAIN; iteration: 5340; epoch: 3; loss: 2.2832589149475098; 
fold: TRAIN; iteration: 5341; epoch: 3; loss: 2.374035596847534; 
fold: TRAIN; iteration: 5342; epoch: 3; loss: 2.300290822982788; 
fold: TRAIN; iteration: 5343; epoch: 3; loss: 2.176379442214966; 
fold: TRAIN; iteration: 5344; epoch: 3; loss: 2.2565298080444336; 
fold: TRAIN; iteration: 5345; epoch: 3; loss: 2.3869712352752686; 
fold: TRAIN; iteration: 5346; epoch: 3; loss: 1.9969210624694824; 
fold: TRAIN; iteration: 5347; epoch: 3; loss: 2.259150743484497; 
fold: TRAIN; iteration: 5348; epoch: 3; loss: 2.0520153045654297; 
fo

fold: TRAIN; iteration: 5458; epoch: 3; loss: 2.1360106468200684; 
fold: TRAIN; iteration: 5459; epoch: 3; loss: 2.2507517337799072; 
fold: TRAIN; iteration: 5460; epoch: 3; loss: 2.2910501956939697; 
fold: TRAIN; iteration: 5461; epoch: 3; loss: 2.310136318206787; 
fold: TRAIN; iteration: 5462; epoch: 3; loss: 2.1956586837768555; 
fold: TRAIN; iteration: 5463; epoch: 3; loss: 2.4403936862945557; 
fold: TRAIN; iteration: 5464; epoch: 3; loss: 2.386075735092163; 
fold: TRAIN; iteration: 5465; epoch: 3; loss: 2.1878204345703125; 
fold: TRAIN; iteration: 5466; epoch: 3; loss: 2.212019443511963; 
fold: TRAIN; iteration: 5467; epoch: 3; loss: 2.063716173171997; 
fold: TRAIN; iteration: 5468; epoch: 3; loss: 2.2041268348693848; 
fold: TRAIN; iteration: 5469; epoch: 3; loss: 2.0529260635375977; 
fold: TRAIN; iteration: 5470; epoch: 3; loss: 2.18660044670105; 
fold: TRAIN; iteration: 5471; epoch: 3; loss: 2.277057647705078; 
fold: TRAIN; iteration: 5472; epoch: 3; loss: 2.323221206665039; 
fol

fold: TRAIN; iteration: 5582; epoch: 3; loss: 2.2144875526428223; 
fold: TRAIN; iteration: 5583; epoch: 3; loss: 2.2806026935577393; 
fold: TRAIN; iteration: 5584; epoch: 3; loss: 2.4250307083129883; 
fold: TRAIN; iteration: 5585; epoch: 3; loss: 2.3106837272644043; 
fold: TRAIN; iteration: 5586; epoch: 3; loss: 2.2654001712799072; 
fold: TRAIN; iteration: 5587; epoch: 3; loss: 2.2809479236602783; 
fold: TRAIN; iteration: 5588; epoch: 3; loss: 2.1827392578125; 
fold: TRAIN; iteration: 5589; epoch: 3; loss: 2.1660280227661133; 
fold: TRAIN; iteration: 5590; epoch: 3; loss: 2.0450079441070557; 
fold: TRAIN; iteration: 5591; epoch: 3; loss: 2.2786505222320557; 
fold: TRAIN; iteration: 5592; epoch: 3; loss: 2.3732571601867676; 
fold: TRAIN; iteration: 5593; epoch: 3; loss: 2.3295743465423584; 
fold: TRAIN; iteration: 5594; epoch: 3; loss: 2.055497884750366; 
fold: TRAIN; iteration: 5595; epoch: 3; loss: 2.090789556503296; 
fold: TRAIN; iteration: 5596; epoch: 3; loss: 2.57309627532959; 
fo

In [8]:
docs['_imputations'].delete_many({})

Now we have trained and evaluated several models of various types. This includes multiple interacting models with mutual dependencies. In the case of our own efficient semantic search, and also the attribute predictor, these models are downstream of the image clip model, in the sense that at inference time, clip must be present in order to be able to execute these models. In the case of attribute prediction, the training task was downstream from the 
spacy pipeline for part-of-speech tagging; these tags were used to produce targets for training. However at run-time, the spacy pipeline won't be necessary.

The models which we've added and trained are now ready to go, and when new data is added or updated to the collection, they will automatically process this data, and insert the model outputs into the collection documents.

Here is the complete set of models which exist in the collection:

In [ ]:
docs.list_models()

In [ ]:
del docs.models['conditional_lm']

In [ ]:
m = docs.models['conditional_lm']

In [ ]:
from superduperdb.models.utils import apply_model

In [ ]:
import random
r = list(docs.find(features={'img': 'clip'}).limit(random.randrange(1000)))[-1]
r

In [ ]:
apply_model(m, r, single=True)

In [ ]:
docs.find_one()

Not all of these respond to incoming data, for that we need to specify the `active` argument:

In [ ]:
docs.list_models(active=True)

We can see that these models have processed all documents and their outputs saved:

In [ ]:
docs.find_one()

Now, let's test what happens when we add new data to the collection, by adding the remaining data points from the 
CoCo data set:

In [ ]:
update = [{**r, "update": True} for r in data[-1000:]]
docs.insert_many(update, verbose=True)